In [97]:
import pickle
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.util import ngrams
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

In [125]:
with open("keywords.pkl", "rb") as f:
    keywords = pickle.load(f)        

In [126]:
print(keywords[3])

['water qualiti quantiti', 'water', 'waterbodi', 'water bodi', 'watercours', 'evapor', 'transpir', 'surfac water', 'ground water', 'runoff', 'contamin', 'contamin', 'water use', 'hydrolog', 'hydrolog', 'hydro', 'hydrostat', 'salin', 'blast', 'withdraw', 'flow', 'peak', 'basin', 'inter-basin', 'water qualiti', 'water qualiti test', 'water tabl', 'contain', 'sediment', 'sewer', 'wast', 'wastewat', 'biosolid', 'sludg', 'septag', 'groundwat', 'groundwater-rel', 'aquif', 'streamflow', 'aquat', 'acquat', 'river', 'hydrometr', 'watersh', 'waterfal', 'sea', 'ocean', 'lake', 'pond', 'fjord', 'wadi', 'run', 'reservoir', 'lagoon', 'bay', 'harbor', 'well', 'well-wat', 'surface-wat', 'surfacewat', 'hot spring', 'creek']


In [127]:
labels_list = ['Physical and meteorological environment',
                'Soil and soil productivity',
                'Vegetation',
                'Water quality and quantity',
                'Fish and fish habitat',
                'Wetlands',
                'Wildlife and wildlife habitat',
                'Species at Risk or Species of Special Status and related habitat',
                'Greenhouse gas (GHG) emissions and climate change',
                'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
                'Air emissions',
                'Acoustic environment',
                'Electromagnetism and Corona Discharge',
                'Human occupancy and resource use',
                'Heritage resources',
                'Navigation and navigation safety',
                'Traditional land and resource use',
                'Social and cultural well-being',
                'Human health and aesthetics',
                'Infrastructure and services',
                'Employment and economy',
                'Environmental Obligations',
                'Rights of Indigenous Peoples']

In [128]:
# df = pd.read_csv('./data/ESA_website_ENG_with_VCs_onehot.csv')
df = pd.read_csv('./data/esa_index_with_table_text_no_labels.csv', index_col=0)

In [129]:
df.head(2)

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT ...,-1
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,-1


In [130]:
table_texts[1]

'TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION Field Site No Name Legal Location UTM Coordinates NAD83 Zone 10 LatitudeLongitude DDMMSS Elevation m asl Fish Stream Classification 17 Fish and Wildlife Timing Window Low Risk 27 Riparian Management Area m 127 Open Water Mean Channel Morphology m 3 Fish Species Captured or Observed During Open Water Aquatic Assessment Previously Documented 4 Location of Nearest Known Fish Habitat 56 Beaver Activity Present YesNo Winter 2012 Aquatic Habitat Investigation Results Supplemental Fish Sampling Recommended for 2013 Comments 20112012 Aitken Creek Route CE WC0 Unnamed tributary to Stewart Creek SW 288020 W6M E 620133 N 6203041 55572764N 12143276W 873 m asl S4 or S6 TBD7 RRZ 0 RMZ 30 or 20 Bankfull 11 Wetted Dry Depth Dry No sampling Dry no fish previously documented Approximately 3 km downstream in Stewart Creek No Winter habitat investigations were not conducted due to dry conditions in fall 2011 Se

In [173]:
table_texts = df['text'].tolist()[0:10]

tokenized_table_texts = []
stemmer = PorterStemmer()

for i, table_text in enumerate(table_texts):
    processed_text = word_tokenize(table_text)
    processed_text = [stemmer.stem(w) for w in processed_text if w not in stopwords.words("english")]
    table_ngram_list = []
    for n in range(1, 7):
        table_ngrams = list(ngrams(processed_text, n))
        table_ngram_list.extend([" ".join(table_gram) for table_gram in table_ngrams])

    tokenized_table_texts.append(table_ngram_list)
    
print(tokenized_table_texts[0][0:200])

['tabl', '3', 'summari', 'OF', 'aquat', 'field', 'work', 'and', 'aborigin', 'field', 'studi', 'particip', 'for', 'the', 'project', 'survey', 'date', 'aborigin', 'commun', 'detail', 'juli', '9', '12', '2011', 'blueberri', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'summer', '2011', 'open', 'water', 'aquat', 'assess', 'septemb', '18', 'octob', '10', '2011', 'blueberri', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'fall', '2011', 'open', 'water', 'aquat', 'assess', 'februari', '26', 'march', '12', '2012', 'blueberri', 'river', 'first', 'nation', 'doig', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'kelli', 'lake', 'cree', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'winter', '2012', 'aquat', 'habitat', 'investig', 'tabl 3', '3 summari', 's

In [175]:
print(tokenized_table_texts[0][:10], '\n', tokenized_table_texts[1][:10])

['tabl', '3', 'summari', 'OF', 'aquat', 'field', 'work', 'and', 'aborigin', 'field'] 
 ['tabl', '4', 'summari', 'OF', 'watercours', 'cross', 'along', 'the', 'north', 'montney']


In [176]:
with open('tokenized_table_texts.pkl', 'wb') as f:
    pickle.dump(tokenized_table_texts, f)

In [177]:
with open('tokenized_table_texts.pkl', 'rb') as f:
    tokenized_table_text = pickle.load(f)

In [178]:
for label in labels_list:
    df[f'{label}'] = 0
    # df[f'{label} - Number of Matches'] = 0
    # df[f'{label} Relevance'] = 0

df.head(2)

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT ...,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [185]:
def label_with_keywords(i, table_text, keywords_for_label, label):
    number_of_matches = len([word for word in keywords_for_label if word in table_text])
    df[f'{label}'][i] = number_of_matches

In [187]:
def string_total_sum_match_2_lists(list_1, list_2):
    """
    This function takes two lists of strings and counts the number of total matches between the two lists, including duplicates.
    The function returns the number of total matches.
    """
    # Initialize the count variable.
    count = 0
    # Iterate over the first list.
    for item in list_1:
        # If the item is in the second list, increment the count variable.
        if item in list_2:
            count += 1
    # Return the count variable.
    return count

In [190]:
df.text[0]

'TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT Survey Date Aboriginal Communities Detail July 9 to 12 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mtis Association Summer 2011 open water aquatic assessments September 18 to October 10 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mtis Association Fall 2011 open water aquatic assessments February 26 to March 12 2012 Blueberry River First Nation Doig River First Nation Halfway River First Nation Kelly Lake Cree First Nation McLeod Lake Indian Band North East Mtis Association Winter 2012 aquatic habitat investigations '

In [188]:
for i, table_text in enumerate(tokenized_table_text):
    for keywords_for_label, label in zip(keywords, labels_list):
        number_of_matches = string_total_sum_match_2_lists(table_text, keywords_for_label)
        df[f'{label}'][i] = number_of_matches
        
with open("esa_index_ENG_labeled.pkl", "wb") as f:
    pickle.dump(df, f)

df.head()

<ipython-input-188-94bef19dd2c9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{label}'][i] = number_of_matches


,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT ...,-1,0,0,0,17,15,0,1,0,0,0,0,0,0,0,0,7,14,1,0,0,0,0,26
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,-1,3,2,6,47,76,3,11,16,0,1,0,0,0,0,0,43,22,2,0,1,20,0,23
2,9136,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,18,48.0,5.87,No,nr

In [ ]:
df.to_csv('./mydata/esa_index_keyword_labeled.csv')

In [144]:
with open("esa_index_ENG_labeled.pkl", "rb") as f:
    df = pickle.load(f)

In [148]:
df.iloc[:, 34:59][0:50]

,text,label,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT ...,-1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1
1,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,-1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1,0,1,1,1,1
2,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,-1,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1,1
3,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,-1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1,1
4,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,-1,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,1,1,1
5,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,-1,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1
6,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,-1,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1,0,1,1,1,1
7,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION F...,-1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1
8,TABLE 5 SUMMARY OF WATER QUALITY PARAMETERS AND MEAN CHANNEL CHARACTERISTICS Field Site No Name ...,-1,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
9,TABLE 5 SUMMARY OF WATER QUALITY PARAMETERS AND MEAN CHANNEL CHARACTERISTICS Field Site No Name ...,-1,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [44]:
labeled_df = pd.read_csv('./data/project-1-at-2021-07-09-00-52-898dbd22.csv')

In [45]:
labeled_df

,Unnamed: 0,text,label,id,sentiment,annotator,annotation_id
0,8889,TABLE 8A.2 REASONABLY FORSEEABLE DEVELOPMENTS IDENTIFIED WITHIN THE SPATIAL BOUNDARIES OF THE BEAR CANYON SECTION Map ID s 1 s Title Proponent Description Construction Schedule s 2 s Assessment Type Quantitative or Qualitative Physical Environment and Soils LSA Water Quality and Quantity RSA Air Quality RSA Acoustic Environment RSA Fish and Fish Habitat RSA Wetlands RSA Vegetation RSA Wildlife and Wildlife Habitat RSA HORU RSA Heritage Resources RSA TRLU RSA Community RSA 13 30 79 12 W6M to...,-1,1,"{""choices"": [""Physical and meteorological environment"", ""Soil and soil productivity"", ""Vegetation"", ""Water quality and quantity"", ""Fish and fish habitat"", ""Wetlands"", ""Wildlife and wildlife habitat"", ""Acoustic environment""]}",jacques.thibodeau@cer-rec.gc.ca,1


In [91]:
for i in range(len(labeled_df)):
    chosen_labels = ast.literal_eval(labeled_df.sentiment[i])["choices"]
    for label in labels_list:
        if label in chosen_labels:
            df[f'{label}'][i] = 1

<ipython-input-91-cb7868f289e9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{label}'][i] = 1


In [92]:
df.iloc[:,36:58]

,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations
0,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0
1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1,0,1,1,1
2,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1
3,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1
4,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28886,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,1
28887,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1
28888,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1
28889,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1


In [ ]:
for i in range(len(labeled_df)):
    

In [117]:
str('"choices" ' + str(df.iloc[0, 36:][df.iloc[0, 36:] == 1].index.values.tolist()) + '}').replace('\'', '')

'"choices" [Physical and meteorological environment, Soil and soil productivity, Vegetation, Water quality and quantity, Fish and fish habitat, Wetlands, Wildlife and wildlife habitat, Acoustic environment, Navigation and navigation safety, Traditional land and resource use, Social and cultural well-being, Rights of Indigenous Peoples]}'

In [119]:
type({"choices": ["Physical and meteorological environment", "Soil and oil productivity", "Vegetation", "Water quality and quantity", "Fish and fish habitat", "Wetlands", "Wildlife and wildlife habitat", "Acoustic environment"]})

dict

In [138]:
myDict = {}
myDict["choices"] = df.iloc[0, 36:][df.iloc[0, 36:] == 1].index.values.tolist()

In [139]:
myDict

{'choices': ['Physical and meteorological environment',
  'Soil and soil productivity',
  'Vegetation',
  'Water quality and quantity',
  'Fish and fish habitat',
  'Wetlands',
  'Wildlife and wildlife habitat',
  'Acoustic environment',
  'Navigation and navigation safety',
  'Traditional land and resource use',
  'Social and cultural well-being',
  'Rights of Indigenous Peoples']}

In [141]:
import json

json.dumps(myDict)

'{"choices": ["Physical and meteorological environment", "Soil and soil productivity", "Vegetation", "Water quality and quantity", "Fish and fish habitat", "Wetlands", "Wildlife and wildlife habitat", "Acoustic environment", "Navigation and navigation safety", "Traditional land and resource use", "Social and cultural well-being", "Rights of Indigenous Peoples"]}'